# Meta programming in ASP

This notebook contains some notes and exercises about Meta programming in ASP.

It is based on section 3 of the paper [1].

Before starting, we recommend you to read carefully all that section,
except the part on Guess-and-check, that is much more advanced than the others.

[1] [Kaminski, R., Romero, J., Schaub, T., & Wanko, P. (2023). How to Build Your Own ASP-based System?! Theory Pract. Log. Program., 23(1), 299–361.](https://arxiv.org/pdf/2008.06692.pdf)

## Preliminaries

Consider the following logic program:

In [1]:
%%file example0.lp
{a}.
 b :- a.

Overwriting example0.lp


You can compute its answer sets with this command:

In [ ]:
! clingo example0.lp 0

And you can ground it using option `--mode=gringo`, that generates a ground program in aspif format:

In [3]:
! clingo example0.lp --mode=gringo

asp 1 0 0
1 1 1 1 0 0
1 0 1 2 0 1 1
4 1 a 1 1
4 1 b 1 2
0


The script `aspif_pretty_printer.py` gives us a more readable version of the aspif format:

In [4]:
! python aspif/aspif_pretty_printer.py example0.lp

{ 1 }.
2 :- 1.
#show a : 1.
#show b : 2.


The script also provides option `--text` to replace the program litearls (`1`, `2`, ...) by symbolic atoms when possible:

In [5]:
! python aspif/aspif_pretty_printer.py --text example0.lp

{ a }.
b :- a.


You can try the script with the file `aspif/example.lp`, and investigate the ground instantiation generated by clingo.

In [ ]:
! cat aspif/example.lp

In [ ]:
! python aspif/aspif_pretty_printer.py --text aspif/example.lp

## A simple example


Let us start with a simple program:

In [8]:
%%file example1.lp
{a}.
#show.

Overwriting example1.lp


We reify it with clingo:

In [9]:
! clingo example1.lp --output=reify

atom_tuple(0).
atom_tuple(0,1).
literal_tuple(0).
rule(choice(0),normal(0)).


These facts represent the following rule in aspif format:
```
{ 1 }.
```

If we eliminate the `#show.` directive then the atom `a` will be shown:

In [10]:
%%file example2.lp
{ a }.

Overwriting example2.lp


In [11]:
! clingo example2.lp --output=reify

atom_tuple(0).
atom_tuple(0,1).
literal_tuple(0).
rule(choice(0),normal(0)).
literal_tuple(1).
literal_tuple(1,1).
output(a,1).


As before, the first four facts represent the rule `{ 1 }.`.

The last three:
```
literal_tuple(1).
literal_tuple(1,1).
output(a,1).
``` 
represent the output directive
```
#show a : 1.
```

In the paper we have seen the meta encoding `meta.lp`:

In [12]:
%%file meta.lp

conjunction(B) :- literal_tuple(B),
        hold(L) : literal_tuple(B, L), L > 0;
    not hold(L) : literal_tuple(B,-L), L > 0.

body(normal(B)) :- rule(_,normal(B)), conjunction(B).
body(sum(B,G))  :- rule(_,sum(B,G)),
    #sum { W,L :     hold(L), weighted_literal_tuple(B, L,W), L > 0 ;
           W,L : not hold(L), weighted_literal_tuple(B,-L,W), L > 0 } >= G.

  hold(A) : atom_tuple(H,A)   :- rule(disjunction(H),B), body(B).
{ hold(A) : atom_tuple(H,A) } :- rule(     choice(H),B), body(B).

#show.
#show T : output(T,B), conjunction(B).

Overwriting meta.lp


Here, we start with a simpler version of it.

In [13]:
%%file meta-simple.lp

conjunction(B) :- literal_tuple(B),
        hold(L) : literal_tuple(B, L), L > 0;
    not hold(L) : literal_tuple(B,-L), L > 0.

  hold(A) : atom_tuple(H,A)   :- rule(disjunction(H),normal(B)), conjunction(B).
{ hold(A) : atom_tuple(H,A) } :- rule(     choice(H),normal(B)), conjunction(B).

#show.
#show T : output(T,B), not literal_tuple(B,_).
#show T : output(T,B),     literal_tuple(B,L), L>0, hold(L).

Overwriting meta-simple.lp


This simpler version does not consider weight rules. 
Given this, the rules for `hold(A)` refer to whether a `normal` body `B` holds using the predicate `conjunction`.

This simpler version represents differently the `#show` directives.
It considers separately the two cases that may happen with programs generated by clingo with option `--output=reify`:
* either we have `output(T,B)` and `B` has no literals,
* or we have `output(T,B)` and `B` has a unique positive literal.

We use this representation simply because it is easier to modify it in the exercises.

We can try `meta-simple.lp` with our two previous examples. 

Option `-Wnone` eliminates some warnings generated by clingo.

In [14]:
! clingo example1.lp --output=reify | clingo - meta-simple.lp 0 -Wnone

clingo version 5.5.0
Reading from - ...
Solving...
Answer: 1

Answer: 2

SATISFIABLE

Models       : 2
Calls        : 1
Time         : 0.004s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.004s


In [15]:
! clingo example2.lp --output=reify | clingo - meta-simple.lp 0 -Wnone

clingo version 5.5.0
Reading from - ...
Solving...
Answer: 1

Answer: 2
a
SATISFIABLE

Models       : 2
Calls        : 1
Time         : 0.004s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.004s


### Exercise 1. 
In the next cell, write a modification of `meta-simple.lp` that interprets the choice rules as normal rules.


In [16]:
%%file meta-simple-ex1.lp

conjunction(B) :- literal_tuple(B),
        hold(L) : literal_tuple(B, L), L > 0;
    not hold(L) : literal_tuple(B,-L), L > 0.

  hold(A) : atom_tuple(H,A)   :- rule(disjunction(H),normal(B)), conjunction(B).
{ hold(A) : atom_tuple(H,A) } :- rule(     choice(H),normal(B)), conjunction(B).

#show.
#show T : output(T,B), not literal_tuple(B,_).
#show T : output(T,B),     literal_tuple(B,L), L>0, hold(L).

Overwriting meta-simple-ex1.lp


Once you have modified the program, the next cell should return a unique answer set that contains the atom `a`.

In [17]:
! clingo example2.lp --output=reify | clingo - meta-simple-ex1.lp 0 -Wnone

clingo version 5.5.0
Reading from - ...
Solving...
Answer: 1

Answer: 2
a
SATISFIABLE

Models       : 2
Calls        : 1
Time         : 0.003s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.003s


Now, modify the `#show` statements of `meta-simple-ex1.lp` in such a way that atoms `T` are shown as `true(T)`.

Once this is done, running the previous cell you should obtain a unique answer set with `true(a)`.

## Extending the example

Consider now the following logic program:

In [18]:
%%file example3.lp
{a}.
 b :- a.

Overwriting example3.lp


In [19]:
! clingo example3.lp --output=reify

atom_tuple(0).
atom_tuple(0,1).
literal_tuple(0).
rule(choice(0),normal(0)).
atom_tuple(1).
atom_tuple(1,2).
literal_tuple(1).
literal_tuple(1,1).
rule(disjunction(1),normal(1)).
output(a,1).
literal_tuple(2).
literal_tuple(2,2).
output(b,2).


As before, the first four facts represent the choice rule `{ 1 }`.

The next ones
```
atom_tuple(1).
atom_tuple(1,2).
literal_tuple(1).
literal_tuple(1,1).
rule(disjunction(1),normal(1)).
```
represent the normal rule:
```
2 :- 1.
```

The fact
```
output(a,1).
```
together with the previous facts 
```
literal_tuple(1).
literal_tuple(1,1).
```
represents the output directive
```
#show a : 1.
```

And 
```
literal_tuple(2).
literal_tuple(2,2).
output(b,2).
```
represent the output directive
```
#show b : 2.
```

We compute the answer sets of `example3.lp`.

From now on, we use option `-V0` to shorten the output.

Note that an empty line represents the empty answer set,
like in this case, where the first line represents such an empty answer set 
and the second line represents the answer set `{a,b}`.

In [20]:
! clingo example3.lp --output=reify | clingo - meta-simple.lp 0 -V0


a b
SATISFIABLE


### Exercise 2. 
In the next cell, write a modification of `meta-simple.lp` that:
* interprets choice rules as normal rules, and normal rules as choice rules, and
* interprets positive literals in the body as negative literals, and negative literals in the body as positive literals.

Under this interpretation, `example3.lp` becomes:
```
a.
{b} :- not a.
```
and leads to the unique answer set `{a}`.

In [21]:
%%file meta-simple-ex2.lp

conjunction(B) :- literal_tuple(B),
        hold(L) : literal_tuple(B, L), L > 0;
    not hold(L) : literal_tuple(B,-L), L > 0.

  hold(A) : atom_tuple(H,A)   :- rule(disjunction(H),normal(B)), conjunction(B).
{ hold(A) : atom_tuple(H,A) } :- rule(     choice(H),normal(B)), conjunction(B).

#show.
#show T : output(T,B), not literal_tuple(B,_).
#show T : output(T,B),     literal_tuple(B,L), L>0, hold(L).

Overwriting meta-simple-ex2.lp


The next cell should return the unique answer set `{a}`.

In [22]:
! clingo example3.lp --output=reify | clingo - meta-simple-ex2.lp 0 -V0


a b
SATISFIABLE


### Exercise 3. 
In the next cell, write a modification of `meta-simple.lp` such that:
* if a rule has some literals in its body, then the body is interpreted as a disjunction of literals, and *not* as a conjunction of literals.

Consider the program `example4.lp`:

In [23]:
%%file example4.lp
{a;b}.
c :- a,b.

Overwriting example4.lp


Under this interpretation, this program becomes:
```
{a;b}.
c :- a.
c :- b.
```
and leads to four answer sets: `{}`, `{a,c}`, `{b,c}`, and `{a,b,c}`.

Note that the modification should only apply to bodies that contain some literal.

In [24]:
%%file meta-simple-ex3.lp

conjunction(B) :- literal_tuple(B),
        hold(L) : literal_tuple(B, L), L > 0;
    not hold(L) : literal_tuple(B,-L), L > 0.

  hold(A) : atom_tuple(H,A)   :- rule(disjunction(H),normal(B)), conjunction(B).
{ hold(A) : atom_tuple(H,A) } :- rule(     choice(H),normal(B)), conjunction(B).

#show.
#show T : output(T,B), not literal_tuple(B,_).
#show T : output(T,B),     literal_tuple(B,L), L>0, hold(L).

Overwriting meta-simple-ex3.lp


The next cell should return the answer sets `{}`, `{a,c}`, `{b,c}`, and `{a,b,c}`.

In [25]:
! clingo example4.lp --output=reify | clingo - meta-simple-ex3.lp 0 -V0

a b c

b
a
SATISFIABLE


## An issue

Let us go back to Exercise 2, and consider the program `example5.lp`:

In [26]:
%%file example5.lp
a.
b :- not a.
#external a.

Overwriting example5.lp


In Exercise 2, this should be interpreted as:
```
{a}.
{b} :- a.
```
and lead to three answer sets: `{}`, `{a}`, and `{a,b}`.

Let's see what do we obtain with the meta-encoding of Exercise 2:

In [ ]:
! clingo example5.lp --output=reify | clingo - meta-simple-ex2.lp 0 -V0

As you should see, using the meta-encoding of Exercise 2 we still obtain the unique answer set `{a}`: how is this possible? 

The warnings printed by clingo give us a clue on this.
They tell us that the atoms of `literal_tuple(B,L)` do not occur in any head.

Let us check the result of the reification of `example5.lp`:

In [ ]:
! clingo example5.lp --output=reify

This corresponds to:
```
1.
#show a.
```
Observe that there is a fact `literal_tuple(0)` but no facts `literal_tuple(0,_)`. 
In this way, `0` refers to the empty body.

What happened here? 
The grounder component of clingo first prints the fact `a.` as `1.`
Then it handles the rule `b :- not a`.
But instead of generating `2 :- -1`, it knows that `a` is a fact, and therefore 
the body of the rule will never be applicable, so it simply skips the rule.
Then, when printing the `#show` directives, the grounder knows that `a` is a fact and prints it directly as 
`#show a.`

Observe how, in the reified program, there is no trace of the original rule `b :- not a`.
Hence, there is nothing we can do in our meta-encoding to interpret this rule differently.

There are some ways to overcome this issue, using `#external` directives, for example.
But we will not dwelve on those for now.

We just have to remember that
* **the meta-encoding is applied to the result of grounding**.

## Weight rules

Consider the program `example6.lp`:

In [ ]:
%%file example6.lp
{a;b;c;d}.
e :- a, 30 #sum{ 20:b; 30:c; 40:d }.
:- not e.

You can reify it using the following command, but instead of doing that, please jump over the cell and have a look at the commented version below.

In [ ]:
! clingo example6.lp --output=reify

In [ ]:
%%file example6-reified.lp

% { 1; 2; 3; 4}.
atom_tuple(0).
atom_tuple(0,1).
atom_tuple(0,2).
atom_tuple(0,3).
atom_tuple(0,4).
literal_tuple(0).
rule(choice(0),normal(0)).

% 5 :- 30 { 20:2; 30:3; 40:4 }.
atom_tuple(1).
atom_tuple(1,5).
weighted_literal_tuple(0).
weighted_literal_tuple(0,2,20).
weighted_literal_tuple(0,3,30).
weighted_literal_tuple(0,4,40).
rule(disjunction(1),sum(0,30)).

% 6 :- 1, 5.
atom_tuple(2).
atom_tuple(2,6).
literal_tuple(1).
literal_tuple(1,5).
literal_tuple(1,1).
rule(disjunction(2),normal(1)).

% :- not 6.
atom_tuple(3).
literal_tuple(2).
literal_tuple(2,-6).
rule(disjunction(3),normal(2)).

% #show a : 1.
literal_tuple(3).
literal_tuple(3,1).
output(a,3).

% #show b : 2.
literal_tuple(4).
literal_tuple(4,2).
output(b,4).

% #show c : 3.
literal_tuple(5).
literal_tuple(5,3).
output(c,5).

% #show d : 4.
literal_tuple(6).
literal_tuple(6,4).
output(d,6).

% #show e : 6.
literal_tuple(7).
literal_tuple(7,6).
output(e,7).

Observe how the rule 
```
e :- a, 30 #sum{ 20:b; 30:c; 40:d }.
```
is separated in two parts.

One part defines the aggregate using an auxiliary atom numbered `5`:
```
5 :- 30 { 20:2; 30:3; 40:4 }.
```
and another handles the rule using that auxiliary atom:
```
6 :- 1, 5.
```


### Exercise 4.

In the next cell, modify the meta-encoding to fix the weights of weight rules to the value of `10`.

Under this interpretation, program `example6.lp` becomes:
```
{a;b;c;d}.
e :- a, 30 #sum{ 10:b; 10:c; 10:d }.
:- not e.
```
and has a unique answer set `{a,b,c,d,e}`.

In [ ]:
%%file meta-ex4.lp 

conjunction(B) :- literal_tuple(B),
        hold(L) : literal_tuple(B, L), L > 0;
    not hold(L) : literal_tuple(B,-L), L > 0.

body(normal(B)) :- rule(_,normal(B)), conjunction(B).
body(sum(B,G))  :- rule(_,sum(B,G)),
    #sum { W,L :     hold(L), weighted_literal_tuple(B, L,W), L > 0 ;
           W,L : not hold(L), weighted_literal_tuple(B,-L,W), L > 0 } >= G.

  hold(A) : atom_tuple(H,A)   :- rule(disjunction(H),B), body(B).
{ hold(A) : atom_tuple(H,A) } :- rule(     choice(H),B), body(B).

#show.
#show T : output(T,B), conjunction(B).

The following command should lead to the unique answer set `{a,b,c,d,e`}.

In [ ]:
! clingo example6.lp --output=reify | clingo - meta-ex4.lp 0 -V0

To finish the exercise, modify again the meta-encoding in such a way that the lower bound for weight rules is always `100`. One this is done, the previous cell should return no answer set.

## Relaxing integrity constraints

A logic program may have zero, one or many answer sets. 
If the program has zero answer sets, we may want to recover part of its *partial* answer sets. 

This is specially useful whenever we are debugging an incorrect program 
that has no answer sets, since in this case the `UNSATISFIABLE` result provided by clingo
tells us nothing about why that program is unsatisfiable.

One way of recovering *partial* answer sets of an unsatisfiable logic program 
is to relax the integrity constraints of the program.
Instead of requiring that no integrity constraint is violated, 
we require that the least number of integrity constraints is violated.

For example, consider the program `example7.lp`, that has no answer sets:

In [ ]:
%%file example7.lp
{ a(1); a(2) }.
:- 1 { a(X) }.
:-   { a(X) } 1.
#show a/1.

In [ ]:
! clingo example7.lp -V0

We would like to intepret the program as this one:

In [ ]:
%%file example7-relaxed.lp
{ a(1); a(2) }.
violated(ic1) :- 1 { a(X) }.
violated(ic2) :-   { a(X) } 1.
:~ violated(R). [1,R]
#show a/1.
#show violated/1.

This gives us two optimal solutions.

The `violated(R)` atoms indicate which integrity constraints where violated.

As can be seen, each optimal solution violated exactly one integrity constraint.

Option `--opt-mode=optN` tells clingo to enumerate all optimal solutions.

In [ ]:
! clingo example7-relaxed.lp 0 --opt-mode=optN

### Exercise 5.

In the next cell, modify `meta.lp` to relax the integrity constraints, as we did in the previous example:

In [ ]:
%%file meta-relaxed.lp

conjunction(B) :- literal_tuple(B),
        hold(L) : literal_tuple(B, L), L > 0;
    not hold(L) : literal_tuple(B,-L), L > 0.

body(normal(B)) :- rule(_,normal(B)), conjunction(B).
body(sum(B,G))  :- rule(_,sum(B,G)),
    #sum { W,L :     hold(L), weighted_literal_tuple(B, L,W), L > 0 ;
           W,L : not hold(L), weighted_literal_tuple(B,-L,W), L > 0 } >= G.

  hold(A) : atom_tuple(H,A)   :- rule(disjunction(H),B), body(B).
{ hold(A) : atom_tuple(H,A) } :- rule(     choice(H),B), body(B).

#show.
#show T : output(T,B), conjunction(B).

The following command should return two optimal solutions, like above with `example7-relaxed.lp`:

In [ ]:
! clingo example7.lp --output=reify | clingo - meta-relaxed.lp 0 --opt-mode=optN